In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

In [2]:
from feature_extraction import ConventionalFeatures, MurugaiahFeatures

In [1]:
a, b, c = [1, 2, 3]

In [2]:
a

1

In [3]:
b

2

In [4]:
c

3